# Team 1
# Automated Detection of Autism Spectrum Disorder Using a Convolutional Neural Network

## Mihir_Ingole_1001944724
## Eldho_Joy_1001986525
## Dannasri_Srinivasan_1001698730

### Turn the below block format to code to download the dataset instead of loading.

In [1]:
from numpy import save, load

abide_loaded = load('abide_dataset.npy',allow_pickle=True)
y_loaded = load('labels.npy',allow_pickle=True)

In [2]:
measure = 'correlation'
y_loaded[y_loaded==2] = 0

from nilearn.connectome import ConnectivityMeasure
from nilearn.connectome import sym_matrix_to_vec

conn_est = ConnectivityMeasure(kind = measure)
conn_matrices = conn_est.fit_transform(abide_loaded)
X = sym_matrix_to_vec(conn_matrices)

In [3]:
import numpy as np
from numpy import save
from numpy import load
import tensorflow as tf
from tensorflow import keras
from keras.layers import Concatenate, Input, Conv2D, Dense, concatenate
from keras.models import Model
from sklearn.model_selection import train_test_split
import os

print(tf.__version__)

2.9.0


In [4]:
from random import randrange
from random import seed

def creatingKFolds(dataset,k_folds):
    '''
    This function implements the k fold cross validation algorithm.
    Input: entire dataset with class labels and value of k.
    Output: list containing k divisions of the entire dataset.
    '''
    
    datasetCopy = list(dataset)
    fold_size = int(len(datasetCopy)/k_folds)
    datasetWithFolds = list()
    for _ in range(k_folds):
        fold = []
        while fold_size > len(fold):
            fold.append(datasetCopy.pop(randrange(len(datasetCopy))))
        datasetWithFolds.append(fold)
        
    return datasetWithFolds

class earlyStoppingRepetition(keras.callbacks.Callback):
    def __init__(self):
        self.acc_metrics = list()
    def on_epoch_end(self, epoch, logs):
        self.acc_metrics.append(logs.get('accuracy') + 0.5)
        if epoch > 4:
            evalfunc = self.acc_metrics[-1]+self.acc_metrics[-2]+self.acc_metrics[-3]
            evalfunc = evalfunc/(3*self.acc_metrics[-1])
            if evalfunc == 1:
                print('\nLoss no longer changing. Hence stopping the training!\n')
                self.model.stop_training = True

class EarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if(logs.get('accuracy') >= 0.9):
            print('Ending training since required accuracy achieved.')
            self.model.stop_training = True

In [5]:
dataset = list()
for i in range(len(list(conn_matrices))):
    temp = list()
    temp.append(conn_matrices[i])
    temp.append(y_loaded[i])
    dataset.append(temp)
    
datasetWithKFolds = creatingKFolds(dataset,10)

In [6]:
def layer_setup():
    inp = Input(shape=(392,392,1))

    model1 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (1,392))(inp)
    model1 = tf.keras.layers.Activation('tanh')(model1)
    model1 = tf.keras.layers.MaxPooling2D(1,400)(model1)


    model2 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (2,392))(inp)
    model2 = tf.keras.layers.Activation('tanh')(model2)
    model2 = tf.keras.layers.MaxPooling2D(1,400)(model2)


    model3 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (3,392))(inp)
    model3 = tf.keras.layers.Activation('tanh')(model3)
    model3 = tf.keras.layers.MaxPooling2D(1,400)(model3)


    model4 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (4,392))(inp)
    model4 = tf.keras.layers.Activation('tanh')(model4)
    model4 = tf.keras.layers.MaxPooling2D(1,400)(model4)


    model5 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (5,392))(inp)
    model5 = tf.keras.layers.Activation('tanh')(model5)
    model5 = tf.keras.layers.MaxPooling2D(1,400)(model5)


    model6 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (6,392))(inp)
    model6 = tf.keras.layers.Activation('tanh')(model6)
    model6 = tf.keras.layers.MaxPooling2D(1,400)(model6)


    model7 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (7,392))(inp)
    model7 = tf.keras.layers.Activation('tanh')(model7)
    model7 = tf.keras.layers.MaxPooling2D(1,400)(model7)

    return inp, model1, model2, model3, model4, model5, model6, model7

In [7]:
def model_setup(inp, model1, model2, model3, model4, model5, model6, model7):
    model = tf.keras.layers.concatenate([model1, model2, model3, model4, model5, model6, model7], axis=1)
    model = tf.keras.layers.Dropout(0.25)(model)
    model = tf.keras.layers.Flatten()(model)
    model = tf.keras.layers.Dense(2800, activation='tanh')(model)
    model = tf.keras.layers.Dense(2, activation='softmax')(model)
    model = tf.keras.Model(inputs=inp, outputs=model)

    return model

In [8]:
def model_compile(model):
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
#model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=0.005), metrics=['accuracy'])

In [9]:
#https://debugah.com/%E3%80%90tensorflow%E3%80%91internalerror-failed-copying-input-tensor-21264/

from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session
import gc

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it does something you should see a number as output

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.compat.v1.Session(config=config))

In [10]:
accuracy_metrics = list()

model = True

for ifold in range(len(datasetWithKFolds)):

    reset_keras()
    inp, model1, model2, model3, model4, model5, model6, model7 = layer_setup()
    
    if model:
        del model
        
    model = model_compile(model_setup(inp, model1, model2, model3, model4, model5, model6, model7))
    
    #callbacks = EarlyStopping()

    completeDataset = list(datasetWithKFolds)
    testFold = completeDataset.pop(ifold)

    trainFolds = list()
    for elem in completeDataset:
        trainFolds += elem

    data_test = list()
    labels_test = list()
    for testElem in testFold:
        data_test.append(testElem[0])
        labels_test.append(testElem[1])

    data_train = list()
    labels_train = list()
    for trainElem in trainFolds:
        data_train.append(trainElem[0])
        labels_train.append(trainElem[1])

    data_train = np.array(data_train)
    labels_train = np.array(labels_train)
    data_test = np.array(data_test)
    labels_test = np.array(labels_test)

    model.fit(x=data_train,y=labels_train,batch_size=32,epochs=300)
    accuracy_metrics.append(model.evaluate(data_test, labels_test)[1])

accuracy = sum(accuracy_metrics)/len(accuracy_metrics)
print("Final model accuracy: ",accuracy)

0
Epoch 1/300
25/25 [==============================] - 15s 191ms/step - loss: 9.1172 - accuracy: 0.5070
Epoch 2/300
25/25 [==============================] - 4s 159ms/step - loss: 2.5560 - accuracy: 0.4840
Epoch 3/300
25/25 [==============================] - 4s 159ms/step - loss: 0.8699 - accuracy: 0.5019
Epoch 4/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9035 - accuracy: 0.5185
Epoch 5/300
25/25 [==============================] - 4s 158ms/step - loss: 0.8503 - accuracy: 0.5032
Epoch 6/300
25/25 [==============================] - 4s 159ms/step - loss: 0.8858 - accuracy: 0.5109
Epoch 7/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9039 - accuracy: 0.4943
Epoch 8/300
25/25 [==============================] - 4s 158ms/step - loss: 0.8972 - accuracy: 0.4968
Epoch 9/300
25/25 [==============================] - 4s 158ms/step - loss: 0.7842 - accuracy: 0.5223
Epoch 10/300
25/25 [==============================] - 4s 159ms/step - loss: 0.9559 - acc

25/25 [==============================] - 4s 158ms/step - loss: 2.6260e-05 - accuracy: 1.0000
Epoch 159/300
25/25 [==============================] - 4s 158ms/step - loss: 2.8019e-05 - accuracy: 1.0000
Epoch 160/300
25/25 [==============================] - 4s 158ms/step - loss: 2.5202e-05 - accuracy: 1.0000
Epoch 161/300
25/25 [==============================] - 4s 158ms/step - loss: 2.1109e-05 - accuracy: 1.0000
Epoch 162/300
25/25 [==============================] - 4s 158ms/step - loss: 2.5485e-05 - accuracy: 1.0000
Epoch 163/300
25/25 [==============================] - 4s 158ms/step - loss: 2.6484e-05 - accuracy: 1.0000
Epoch 164/300
25/25 [==============================] - 4s 158ms/step - loss: 2.0609e-05 - accuracy: 1.0000
Epoch 165/300
25/25 [==============================] - 4s 158ms/step - loss: 2.3479e-05 - accuracy: 1.0000
Epoch 166/300
25/25 [==============================] - 4s 159ms/step - loss: 2.0151e-05 - accuracy: 1.0000
Epoch 167/300
25/25 [==============================

25/25 [==============================] - 4s 159ms/step - loss: 0.9886 - accuracy: 0.5185
Epoch 11/300
25/25 [==============================] - 4s 158ms/step - loss: 0.8443 - accuracy: 0.4943
Epoch 12/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9826 - accuracy: 0.5134
Epoch 13/300
25/25 [==============================] - 4s 158ms/step - loss: 0.8321 - accuracy: 0.5032
Epoch 14/300
25/25 [==============================] - 4s 158ms/step - loss: 1.5448 - accuracy: 0.4994
Epoch 15/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9168 - accuracy: 0.5019
Epoch 16/300
25/25 [==============================] - 4s 159ms/step - loss: 0.7950 - accuracy: 0.5377
Epoch 17/300
25/25 [==============================] - 4s 158ms/step - loss: 0.8131 - accuracy: 0.5556
Epoch 18/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9628 - accuracy: 0.5147
Epoch 19/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9453 - accuracy: 

25/25 [==============================] - 4s 158ms/step - loss: 1.2551 - accuracy: 0.5019
Epoch 170/300
25/25 [==============================] - 4s 158ms/step - loss: 0.8039 - accuracy: 0.5070
Epoch 171/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9969 - accuracy: 0.5338
Epoch 172/300
25/25 [==============================] - 4s 159ms/step - loss: 1.0163 - accuracy: 0.4713
Epoch 173/300
25/25 [==============================] - 4s 158ms/step - loss: 1.0571 - accuracy: 0.5377
Epoch 174/300
25/25 [==============================] - 4s 158ms/step - loss: 1.0060 - accuracy: 0.4700
Epoch 175/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9488 - accuracy: 0.5070
Epoch 176/300
25/25 [==============================] - 4s 159ms/step - loss: 0.7963 - accuracy: 0.5300
Epoch 177/300
25/25 [==============================] - 4s 158ms/step - loss: 0.7671 - accuracy: 0.4917
Epoch 178/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9704 - a

25/25 [==============================] - 4s 160ms/step - loss: 0.7839 - accuracy: 0.5364
Epoch 27/300
25/25 [==============================] - 4s 159ms/step - loss: 1.0024 - accuracy: 0.5070
Epoch 28/300
25/25 [==============================] - 4s 160ms/step - loss: 0.9746 - accuracy: 0.4904
Epoch 29/300
25/25 [==============================] - 4s 160ms/step - loss: 0.8447 - accuracy: 0.4904
Epoch 30/300
25/25 [==============================] - 4s 160ms/step - loss: 0.7443 - accuracy: 0.5211
Epoch 31/300
25/25 [==============================] - 4s 159ms/step - loss: 0.9753 - accuracy: 0.5019
Epoch 32/300
25/25 [==============================] - 4s 159ms/step - loss: 0.9892 - accuracy: 0.5185
Epoch 33/300
25/25 [==============================] - 4s 159ms/step - loss: 0.8975 - accuracy: 0.5070
Epoch 34/300
25/25 [==============================] - 4s 159ms/step - loss: 0.9889 - accuracy: 0.5479
Epoch 35/300
25/25 [==============================] - 4s 159ms/step - loss: 1.2931 - accuracy: 

...

25/25 [==============================] - 4s 158ms/step - loss: 5.4280e-05 - accuracy: 1.0000
Epoch 156/300
25/25 [==============================] - 4s 158ms/step - loss: 4.4681e-05 - accuracy: 1.0000
Epoch 157/300
25/25 [==============================] - 4s 158ms/step - loss: 4.2658e-05 - accuracy: 1.0000
Epoch 158/300
25/25 [==============================] - 4s 158ms/step - loss: 4.2611e-05 - accuracy: 1.0000
Epoch 159/300
25/25 [==============================] - 4s 158ms/step - loss: 4.1426e-05 - accuracy: 1.0000
Epoch 160/300
25/25 [==============================] - 4s 158ms/step - loss: 4.8261e-05 - accuracy: 1.0000
Epoch 161/300
25/25 [==============================] - 4s 158ms/step - loss: 4.9345e-05 - accuracy: 1.0000
Epoch 162/300
25/25 [==============================] - 4s 158ms/step - loss: 4.2579e-05 - accuracy: 1.0000
Epoch 163/300
25/25 [==============================] - 4s 158ms/step - loss: 3.7062e-05 - accuracy: 1.0000
Epoch 164/300
25/25 [==============================

25/25 [==============================] - 4s 158ms/step - loss: 0.9796 - accuracy: 0.4968
Epoch 8/300
25/25 [==============================] - 4s 158ms/step - loss: 0.8825 - accuracy: 0.5300
Epoch 9/300
25/25 [==============================] - 4s 158ms/step - loss: 1.0016 - accuracy: 0.4815
Epoch 10/300
25/25 [==============================] - 4s 158ms/step - loss: 0.7861 - accuracy: 0.5326
Epoch 11/300
25/25 [==============================] - 4s 158ms/step - loss: 0.7969 - accuracy: 0.5351
Epoch 12/300
25/25 [==============================] - 4s 158ms/step - loss: 1.1065 - accuracy: 0.5019
Epoch 13/300
25/25 [==============================] - 4s 158ms/step - loss: 1.0192 - accuracy: 0.4802
Epoch 14/300
25/25 [==============================] - 4s 158ms/step - loss: 1.1804 - accuracy: 0.5147
Epoch 15/300
25/25 [==============================] - 4s 158ms/step - loss: 1.9785 - accuracy: 0.4853
Epoch 16/300
25/25 [==============================] - 4s 158ms/step - loss: 1.0817 - accuracy: 0.

25/25 [==============================] - 4s 158ms/step - loss: 7.4736e-05 - accuracy: 1.0000
Epoch 166/300
25/25 [==============================] - 4s 158ms/step - loss: 8.3732e-05 - accuracy: 1.0000
Epoch 167/300
25/25 [==============================] - 4s 158ms/step - loss: 7.4765e-05 - accuracy: 1.0000
Epoch 168/300
25/25 [==============================] - 4s 158ms/step - loss: 6.7228e-05 - accuracy: 1.0000
Epoch 169/300
25/25 [==============================] - 4s 158ms/step - loss: 9.1314e-05 - accuracy: 1.0000
Epoch 170/300
25/25 [==============================] - 4s 158ms/step - loss: 6.6007e-05 - accuracy: 1.0000
Epoch 171/300
25/25 [==============================] - 4s 158ms/step - loss: 6.5402e-05 - accuracy: 1.0000
Epoch 172/300
25/25 [==============================] - 4s 158ms/step - loss: 5.6636e-05 - accuracy: 1.0000
Epoch 173/300
25/25 [==============================] - 4s 158ms/step - loss: 5.3595e-05 - accuracy: 1.0000
Epoch 174/300
25/25 [==============================

25/25 [==============================] - 4s 157ms/step - loss: 0.7435 - accuracy: 0.5377
Epoch 20/300
25/25 [==============================] - 4s 158ms/step - loss: 1.0878 - accuracy: 0.5083
Epoch 21/300
25/25 [==============================] - 4s 158ms/step - loss: 0.8955 - accuracy: 0.5109
Epoch 22/300
25/25 [==============================] - 4s 158ms/step - loss: 1.0137 - accuracy: 0.4994
Epoch 23/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9329 - accuracy: 0.5338
Epoch 24/300
25/25 [==============================] - 4s 158ms/step - loss: 1.1960 - accuracy: 0.5070
Epoch 25/300
25/25 [==============================] - 4s 159ms/step - loss: 0.8494 - accuracy: 0.5172
Epoch 26/300
25/25 [==============================] - 4s 158ms/step - loss: 0.7445 - accuracy: 0.5198
Epoch 27/300
25/25 [==============================] - 4s 158ms/step - loss: 1.1817 - accuracy: 0.5262
Epoch 28/300
25/25 [==============================] - 4s 157ms/step - loss: 0.9936 - accuracy: 

25/25 [==============================] - 4s 158ms/step - loss: 0.0032 - accuracy: 0.9987
Epoch 179/300
25/25 [==============================] - 4s 158ms/step - loss: 0.0118 - accuracy: 0.9962
Epoch 180/300
25/25 [==============================] - 4s 158ms/step - loss: 0.0464 - accuracy: 0.9821
Epoch 181/300
25/25 [==============================] - 4s 158ms/step - loss: 0.4915 - accuracy: 0.8531
Epoch 182/300
25/25 [==============================] - 4s 158ms/step - loss: 0.4120 - accuracy: 0.8429
Epoch 183/300
25/25 [==============================] - 4s 158ms/step - loss: 0.4647 - accuracy: 0.8238
Epoch 184/300
25/25 [==============================] - 4s 159ms/step - loss: 0.5354 - accuracy: 0.8212
Epoch 185/300
25/25 [==============================] - 4s 158ms/step - loss: 0.3094 - accuracy: 0.8851
Epoch 186/300
25/25 [==============================] - 4s 158ms/step - loss: 0.1552 - accuracy: 0.9323
Epoch 187/300
25/25 [==============================] - 4s 158ms/step - loss: 0.2117 - a

In [11]:
print(accuracy_metrics)

[0.5632184147834778, 0.6436781883239746, 0.5632184147834778, 0.5862069129943848, 0.4712643623352051, 0.6781609058380127, 0.5517241358757019, 0.6206896305084229, 0.5057471394538879, 0.6321839094161987]


In [12]:
print(f"Accuracy of the best fold: {max(accuracy_metrics)}")

Accuracy of the best fold: 0.6781609058380127
